In [1]:
from torchvision import datasets
data_path = '../temp'
c10 = datasets.CIFAR10(data_path, train=True, download=True)
c10_val = datasets.CIFAR10(data_path, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
from torchvision import transforms
to_tensor = transforms.ToTensor()
label_map = {0: 0, 2: 1}
c2 = [(to_tensor(img), label_map[label])
                 for img, label in c10
                 if label in [0, 2]]
c2_val = [(to_tensor(img), label_map[label])
                 for img, label in c10_val
                 if label in [0, 2]]

In [9]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [7]:
img = c2[0][0]

In [5]:
model = Net()

In [16]:
img.shape

torch.Size([3, 32, 32])

In [10]:
model(img)

tensor([[0.0839, 0.1400]], grad_fn=<AddmmBackward0>)

In [11]:
loss_fn = nn.CrossEntropyLoss()

In [12]:
imgs = torch.stack([img[0].unsqueeze(0) for img in c2]).squeeze(1)

In [13]:
imgs.shape

torch.Size([10000, 3, 32, 32])

In [32]:
outputs = model(imgs)

In [33]:
outputs.shape

torch.Size([10000, 2])

In [14]:
labels = torch.tensor([img[1] for img in c2])

In [36]:
loss = loss_fn(outputs, labels)

In [37]:
loss

tensor(0.7061, grad_fn=<NllLossBackward0>)

In [39]:
2.3 ** (-0.7)

0.5582008773935958

In [16]:
trainer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
for i in range(100):
    outputs = model(imgs)
    loss = loss_fn(outputs, labels)
    
    trainer.zero_grad()
    loss.backward()
    trainer.step()
    
    if i%5 == 0:
        print(i, loss, evaluate_train(outputs), evaluate_val())

0 tensor(0.6573, grad_fn=<NllLossBackward0>) 0.6851 0.6975
5 tensor(0.6553, grad_fn=<NllLossBackward0>) 0.6899 0.7
10 tensor(0.6532, grad_fn=<NllLossBackward0>) 0.695 0.703
15 tensor(0.6510, grad_fn=<NllLossBackward0>) 0.6986 0.709
20 tensor(0.6486, grad_fn=<NllLossBackward0>) 0.7047 0.7125
25 tensor(0.6461, grad_fn=<NllLossBackward0>) 0.7085 0.7165
30 tensor(0.6436, grad_fn=<NllLossBackward0>) 0.7117 0.721
35 tensor(0.6408, grad_fn=<NllLossBackward0>) 0.7144 0.7215
40 tensor(0.6380, grad_fn=<NllLossBackward0>) 0.7179 0.7245
45 tensor(0.6350, grad_fn=<NllLossBackward0>) 0.7207 0.7245
50 tensor(0.6319, grad_fn=<NllLossBackward0>) 0.7239 0.7265
55 tensor(0.6287, grad_fn=<NllLossBackward0>) 0.7277 0.7305
60 tensor(0.6253, grad_fn=<NllLossBackward0>) 0.7312 0.7315
65 tensor(0.6218, grad_fn=<NllLossBackward0>) 0.7329 0.736
70 tensor(0.6182, grad_fn=<NllLossBackward0>) 0.7359 0.74


In [23]:
labels_val = torch.tensor([img[1] for img in c2_val])
imgs_val = torch.stack([img[0].unsqueeze(0) for img in c2_val]).squeeze(1)

In [28]:
def evaluate_val():
    pred = model(imgs_val)
    _, p = torch.max(pred, dim=1)
    return int((labels_val == p).sum())/2000

In [29]:
evaluate_val()

0.6965

In [39]:
def evaluate_train(pred):
    _, p = torch.max(pred, dim=1)
    return int((labels == p).sum())/10000

In [40]:
outputs = model(imgs)

In [41]:
evaluate_train(outputs)

0.6851